In [3]:
!pip install bs4

import bs4
from bs4 import BeautifulSoup
import requests

#Getting HTML content for a given url
def get_soup(url):
  try:
    r = requests.get(url)
  except ConnectionError:
    print(f"Connection error for {url}")
  #print(r.content)

  #print(soup.prettify())
  return BeautifulSoup(r.content, 'lxml')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
bookmark_soup = BeautifulSoup(open("/content/Sample_Bookmark.html", encoding="utf8"), "html.parser")

url_list=[]
for link in bookmark_soup.findAll('a'):
    url_list.append(link.get('href'))

In [5]:
url_soup = get_soup(url_list[0])
for script in url_soup(['script', 'style']):
    script.extract()
training_source = url_soup.get_text(' ', strip=True)
training_source

'Interactive Q&A Bot with RASA + Haystack + RoBERTa | by saritapadoshi | Medium Get unlimited access Open in app Home Notifications Lists Stories Write saritapadoshi Follow May 15, 2021 · 8 min read Interactive Q&A Bot with RASA + Haystack + RoBERTa In this article, I plan to present the steps in creating an interactive bot for ‘Question and Answer’ model with K12 education knowledge base, using pre-trained Hugging Face transformer model ( RoBERTa ), fine tuned with SQUAD 2.0 Q&A data set. The solution also makes use of Haystack framework for document retrieval and reader pipeline creation and Rasa for chat bot front-end framework to keep the interaction natural. Architecture 1. Why BERT Transformer model ? Transformers in general apply attention mechanisms to gather information about the relevant context of words, and then encode that context in a rich vector that smartly represents the word in parallel compared to LSTM or RNN. BERT ( Bidirectional Encoder Representations from Transfo

In [6]:
train_text = training_source.split('.')
for i in range(len(train_text)):
  train_text[i] += '.'

In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 19.4 MB/s 
     |████████████████████████████████| 6.6 MB 61.3 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 


In [8]:
import transformers
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch

In [9]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForMaskedLM.from_pretrained("roberta-base")

In [10]:
inputs = tokenizer(train_text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
inputs

{'input_ids': tensor([[    0, 26267, 12228,  ...,     1,     1,     1],
        [    0,   288,  1209,  ...,     1,     1,     1],
        [    0,    20,  2472,  ...,     1,     1,     1],
        ...,
        [    0, 11087, 10690,  ...,     1,     1,     1],
        [    0,  3098,  2094,  ...,     1,     1,     1],
        [    0,  2184,   901,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [11]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [12]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [13]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 0) * \
           (inputs.input_ids != 2) * (inputs.input_ids != 0)

In [14]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [15]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [16]:
class Bookmark_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [17]:
dataset = Bookmark_Dataset(inputs)

In [18]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='out',
    per_device_train_batch_size=8,
    num_train_epochs=10
)

In [19]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset
)

In [20]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 124
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 160
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=160, training_loss=7.626837158203125, metrics={'train_runtime': 184.4269, 'train_samples_per_second': 6.724, 'train_steps_per_second': 0.868, 'total_flos': 326449174487040.0, 'train_loss': 7.626837158203125, 'epoch': 10.0})